##### About Dataset

The loan approval dataset is a collection of financial records and associated information used to determine the eligibility of individuals or organizations for obtaining loans from a lending institution. It includes various factors such as cibil score, income, employment status, loan term, loan amount, assets value, and loan status. This dataset is commonly used in machine learning and data analysis to develop models and algorithms that predict the likelihood of loan approval based on the given features.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("./archive/loan_approval_dataset.csv")

In [ ]:
df.head()

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected


In [ ]:
df.drop("loan_id", axis=1, inplace=True)

In [ ]:
df.isna().sum()

 no_of_dependents            0
 education                   0
 self_employed               0
 income_annum                0
 loan_amount                 0
 loan_term                   0
 cibil_score                 0
 residential_assets_value    0
 commercial_assets_value     0
 luxury_assets_value         0
 bank_asset_value            0
 loan_status                 0
dtype: int64

In [ ]:
df.dtypes

 no_of_dependents             int64
 education                   object
 self_employed               object
 income_annum                 int64
 loan_amount                  int64
 loan_term                    int64
 cibil_score                  int64
 residential_assets_value     int64
 commercial_assets_value      int64
 luxury_assets_value          int64
 bank_asset_value             int64
 loan_status                 object
dtype: object

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()

In [ ]:
for label, content in df.items():
    if pd.api.types.is_object_dtype(content):
        df[label] = enc.fit_transform(content)

In [ ]:
df.dtypes

 no_of_dependents            int64
 education                   int32
 self_employed               int32
 income_annum                int64
 loan_amount                 int64
 loan_term                   int64
 cibil_score                 int64
 residential_assets_value    int64
 commercial_assets_value     int64
 luxury_assets_value         int64
 bank_asset_value            int64
 loan_status                 int32
dtype: object

In [ ]:
X = df.drop(" loan_status", axis=1)
y = df[" loan_status"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=10000)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
feature_importances = pd.Series(model.feature_importances_, model.feature_names_in_)


In [ ]:
import seaborn as sn

In [ ]:
sn.barplot(x=feature_importances.values, y=feature_importances.index, width=1);

In [ ]:
y_preds = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
sn.heatmap(confusion_matrix(y_pred=y_preds, y_true=y_test), linewidths=1, annot=True, fmt="d");

In [ ]:
feature_importances

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
import optuna

In [ ]:
def objective(trial):
    params = {
    "learning_rate": trial.suggest_float("learning_rate", 0, 1),
    "max_depth": trial.suggest_int("max_depth", 8, 12),
    "max_leaves": trial.suggest_int("max_leaves", 2, 8),
    "num_parallel_tree": trial.suggest_int("num_parallel_tree", 2, 4),
    "n_estimators": trial.suggest_int("n_estimators", 200, 1000)
}
    model = XGBClassifier(**params)
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)


In [ ]:
study = optuna.create_study(direction="maximize")

In [ ]:
study.optimize(objective, n_trials=100, n_jobs=-1, show_progress_bar=True)

In [ ]:
study.best_params

In [ ]:
new_params = study.best_params

In [ ]:
model = XGBClassifier(**new_params)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
optuna.visualization.plot_optimization_history(study)